# I - Theroetical Calculations

As always, we will begin by writing down the classical Hamiltonian of the circuit before moving on to operators.
Here we have : 

$$H = \frac 1 {2C} q^2 - E_{J1} \cos(\varphi_1) - E_{J2} \cos(\varphi_2)$$

Since we have a loop, we must take into account the external flux, which links the two fluxes through the following relationship :
$$\varphi_1 -\varphi_2 = \varphi_{ext}$$

Let's introduce de mean of the fluxes $\varphi = \frac {\varphi_1 + \varphi_2} 2$, which leads us to the relationships :
$$
\begin{equation}
    \begin{cases}
        \varphi_1 = \varphi + \frac {\varphi_{ext}} 2 \\
        \varphi_2 = \varphi - \frac {\varphi_{ext}} 2
    \end{cases}\,.
\end{equation}
$$

We can now rewrite the Hamiltonian as :

$$
H = \frac 1 {2C} q^2 - E_{J1} \cos(\varphi + \frac {\varphi_{ext}} 2) - E_{J2} \cos(\varphi - \frac {\varphi_{ext}} 2) 
$$

One can then show that :
$$
H = 4E_C n^2 - E_{J}(\varphi_{ext}) \cos(\varphi - \varphi_0)
$$

Where $E_{J\Sigma} = E_{J1} + E_{J2}$, $\varphi_0 = d\tan(\frac{\varphi_{ext}}2)$ and $d = \frac {E_{J2}- E_{J1}}{E_{J\Sigma}}$, and finally :
$$
E_{J}(\varphi_{ext}) = E_{J\Sigma}\cos\left(\frac {\varphi_{ext}} 2\right) \sqrt{1+\varphi_0^2}
$$

In terms of operators, we then get : 
$$
\hat H = 4E_C \hat n^2 - E_{J}(\varphi_{ext}) \cos(\hat\varphi - \varphi_0)
$$

If we do not consider the circuit to be capacitively shunted, we must also take into account $n_g$ :
$$
\hat H = 4E_C (\hat n-n_g)^2 - E_{J}(\varphi_{ext}) \cos(\hat\varphi - \varphi_0)
$$


# II - Implementation using SCQubits

In [2]:
import scqubits as sc

In [3]:
tune_tmon = sc.TunableTransmon(
   EJmax=50.0,
   EC=0.5,
   d=0.01,
   flux=0.0,
   ng=0.0,
   ncut=30
)

In [10]:
evals_sc = tune_tmon.eigenvals()
evals_sc -= evals_sc[0]

# III - Implementation using numpy

Here we jump straight to using Numpy, as the implementation is not very different between QuTiP and Numpy. We must now also compute $\sin(\hat\varphi)$ as we need to obtain $\cos(\hat\varphi - \varphi_0)$

In [6]:
import numpy as np
import numpy.linalg as alg

In [7]:
def hamiltonian_np(EJmax,EC,d,flux=0.0,ng=0.0,truncation_level=31):
    
    dim_transmon = 2 * truncation_level // 2 + 1
    charge = np.zeros((dim_transmon, dim_transmon))
    cos_phi = np.zeros((dim_transmon, dim_transmon))
    sin_phi = np.zeros((dim_transmon, dim_transmon))

    for i in range(dim_transmon - 1):
        cos_phi[i, i + 1] = 1/2
        cos_phi[i + 1, i] = 1/2

        sin_phi[i, i + 1] = -1/2
        sin_phi[i + 1, i] = 1/2

        charge[i, i] = i - dim_transmon // 2

    charge[-1, -1] = dim_transmon // 2

    phi_0 = d*np.tan(flux/2)

    EJ = EJmax * np.cos(flux/2)*np.sqrt(1 + phi_0**2)

    return 4 * EC * (charge - np.eye(dim_transmon))**2 - EJ * (np.cos(phi_0)*cos_phi - np.sin(phi_0)*sin_phi)

In [11]:
evals_np = np.sort(alg.eigvals(hamiltonian_np(EJmax=50.0,EC=0.5,d=0.01,flux=0.0,ng=0.0,truncation_level=30)))
evals_np -= evals_np[0]


In [14]:
print(evals_sc, '\n', evals_np[:6])

[ 0.         13.62256777 26.69738841 39.18380108 51.02888606 62.16364119] 
 [ 0.         13.62256777 26.69738841 39.18380108 51.02888606 62.16364119]
